Purpose: save icons on each card in a specified deck to be manually sorted

In [ ]:
import cv2
import imutils
import os
import matplotlib.pyplot as plt
import numpy as np
from image_processing import contrast_enhancement, find_object_contours

output_directory = "deck-08"  # Directory to save the images
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist
card_directory = "dobble_deck08_cards_55" # the deck we're iterating through

# For dobble deck 2, clipLimit 5.0, sym_card_size_ratio 1/20, binary_thresh 190
# card 35 in deck 2 has a really small splatter that we need to deal with -- it requires 1/25
# card 27 in deck 2's moon is stupidly close to the fire which might fuck with the distance thresholding idea we had
# card 29 in deck 2's daisy is a little screwed up
# For dobble deck 4 card 02, the web and the pencil are very different and it's hard to find a threshold that works for both


clipLimit = 4.0
# This parameter controls the min size a symbol has to be to qualify as an object
# 1/20 would mean the symbol is 1/20th the size of the full card
sym_card_size_ratio = 1/20
binary_thresh = 190

counter = 0 # count how many cards have more/less contours than expected
image_not_found_cnt = 0
for file in os.listdir(card_directory):
  VISUALIZE = False

  # load image
  image = cv2.imread(os.path.join(card_directory, file))
  # this is because my mac is doing stupid stuff with .ds store or smth
  if image is None:
      image_not_found_cnt += 1
      if image_not_found_cnt > 1:
        raise FileNotFoundError(f"Could not load image: {file}")
      else:
        continue

  # enhance the contrast in the lightness color space
  enhanced_RGB = contrast_enhancement(image, clipLimit)
  # draw the smaller contours for visualization
  [card_mask, masked_binary, object_contours] = find_object_contours(enhanced_RGB.copy(), sym_card_size_ratio, binary_thresh)
  contoured_output_image = enhanced_RGB.copy()
  cv2.drawContours(contoured_output_image, object_contours, -1, (0, 255, 0), thickness=1)

  # save each contour
  for i, contour in enumerate(object_contours):
    x, y, w, h = cv2.boundingRect(contour)
    # Crop the ROI from the original image
    roi = enhanced_RGB[y:y+h, x:x+w]
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.drawContours(mask, [contour - [x, y]], -1, 255, thickness=cv2.FILLED)
    roi_masked = cv2.bitwise_and(roi, roi, mask=mask)
    # Convert to BGRA (add alpha channel)
    roi_bgra = cv2.cvtColor(roi_masked, cv2.COLOR_RGB2BGRA)
    roi_bgra[:, :, 3] = mask  # Set alpha channel based on the mask
    # Save the ROI as a PNG with transparency
    contour_filename = f"{file.split('.')[0]}_contour_{i+1}.png"
    contour_path = os.path.join(output_directory, contour_filename)
    cv2.imwrite(contour_path, roi_bgra)
    print(f"Saved transparent contour {i+1} to {contour_path}")


  # if it messed up and found not 8 symbols, visualize them
  # This is usually caused by certain icons (splat, question, exclamation) being split into multiple symbols
  if len(object_contours) != 8:
    counter += 1
    VISUALIZE = True
  
  if VISUALIZE:
    contour_areas = [cv2.contourArea(contour) for contour in object_contours]
    print(f"There are {len(object_contours)} objects detected on {file}, with a minimum size of {min(contour_areas)}")

    fig, axs = plt.subplots(2, 2, figsize=(7, 7))

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    axs[0,0].imshow(image_rgb)
    axs[0,0].set_title(file)
    axs[0,0].axis("off")

    axs[0,1].imshow(enhanced_RGB, cmap="gray")
    axs[0,1].set_title(f'Contrast Enhanced')
    axs[0,1].axis("off")

    axs[1,0].imshow(masked_binary, cmap="gray")
    axs[1,0].set_title(f"Masked Binary")
    axs[1,0].axis("off")

    axs[1,1].imshow(contoured_output_image, cmap="gray")
    axs[1,1].set_title(f'Object Contoured Card')
    axs[1,1].axis("off")

    plt.tight_layout()
    plt.show()

print(counter)